In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from colorcloud.cheng2023TransRVNet import TransVRNet
from colorcloud.cheng2023TransRVNet import TransRVNet_loss
import lightning as L
import numpy as np
import wandb
from lightning.pytorch.loggers import WandbLogger
from datetime import datetime
import torch
import torchvision
from lovasz_losses import lovasz_softmax
from boundary_loss import BoundaryLoss
from colorcloud.behley2019iccv import SemanticKITTIDataset, SphericalProjection, ProjectionTransform, ProjectionToTensorTransform
from torchvision.transforms import v2
from torch.utils.data import DataLoader

/workspace/colorcloud/colorcloud/cheng2023TransRVNet.py:999: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (classes is 'present' and fg.sum() == 0):
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [3]:
data_path = '../../data'
ds = SemanticKITTIDataset(data_path)
frame, label, mask = ds[128]

In [4]:
# inserir data rotation no transform 
# inverter = torchvision.transforms.RandomInvert(p=0.5)
# rotation = torchvision.transforms.RandomRotation(degrees=(0, 180))
# Data augmentation
# ref = inverter(reflectance)
# dep = inverter(depth)
# xyz = inverter(xyz)

proj = SphericalProjection(fov_up_deg=12., fov_down_deg=-26., W=1024, H=64)
tfms = v2.Compose([
    ProjectionTransform(proj),
    ProjectionToTensorTransform(),
])
ds.set_transform(tfms)
ds.set_transform(tfms)
frame_img, label_img, mask_img = ds[128]


In [5]:
bs = 1

data_loader = DataLoader(ds, bs)

## Setup

### Convolutions' Parameters

In [6]:
p1 = {
    "b1_in": 1,
    "b1_out1": 1,
    "b1_out2": 1,
    "b1_out3": 1,
    "b2_in": 32,
    "b2_out": 32,
    "b3_in": 64,
    "b3_out1": 64,
    "b3_out2": 64,
    "b3_out3": 64,
    "output": 128,
}

p2 = {
    "b1_in": 3,
    "b1_out1": 1,
    "b1_out2": 1,
    "b1_out3": 1,
    "b2_in": 32,
    "b2_out": 32,
    "b3_in": 64,
    "b3_out1": 64,
    "b3_out2": 64,
    "b3_out3": 64,
    "output": 128,
}

## Training

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [8]:
model = TransVRNet(p1, p2).to(device)
loss = TransRVNet_loss(device)

/opt/conda/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
dropout = torch.nn.Dropout(p=0.2, inplace=False)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-1, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                                                max_lr=0.002, 
                                                div_factor=1, 
                                                final_div_factor=10, 
                                                steps_per_epoch=len(data_loader), 
                                                epochs=30)

In [10]:
for batch in data_loader:
    img, label, mask = batch
    xyz = img[:, :3, :, :]
    reflectance = img[:, 3, :, :].unsqueeze(1)
    depth = img[:, 4, :, :].unsqueeze(1)
    
    break

In [11]:
epochs = 1
for epoch in range(epochs):
    model.train()
    
    for batch_idx, batch in enumerate(data_loader):
        img, label, mask = batch
        img, label, mask = img.to(device), label.to(device), mask.to(device)
        
        # Separate channels
        xyz = img[:, :3, :, :]
        reflectance = img[:, 3, :, :].unsqueeze(1)
        depth = img[:, 4, :, :].unsqueeze(1)
        
        optimizer.zero_grad()

        # Forward pass
        output = model(reflectance, depth, xyz)
        
        # Apply dropout
        output = dropout(output)
        loss = loss(output, label)
        
        loss.backward()
        
        optimizer.step()
        
        scheduler.step()
        
        if batch_idx % log_interval == 0:
            print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}")
    
        print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}")

torch.save(model.state_dict(), "trained_transvrnet_model.pth")

/opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [0,0,0], thread: [800,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [0,0,0], thread: [801,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [0,0,0], thread: [802,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [0,0,0], thread: [803,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cuda/NLLLoss2d.cu:106: nll_loss2d_forward_kernel: block: [0,0,0], thread: [804,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/cond

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
